In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sklearn.metrics import mean_squared_error

In [ ]:
# resuable code

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Function for frequent use for houlrly data
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def daily_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable, model:list):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"daily": variable,
	"models": model
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_sum = daily.Variables(0).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
	)}
	daily_data["open_meteo"] = daily_sum

	daily_dataframe = pd.DataFrame(data = daily_data)
	if daily_dataframe.empty:
		return print("The daily limit reached")
	return daily_dataframe

def calculate_accuracy(original_data, predicted_data):
	mbe = np.mean(predicted_data - original_data)
	rmse = np.sqrt(mean_squared_error(original_data, predicted_data))
	correlation = np.corrcoef(original_data, predicted_data)[0, 1]

	# Print results
	print(f"Mean Bias Error (MBE): {mbe}")
	print(f"Root Mean Square Error (RMSE): {rmse}")
	print(f"Correlation: {correlation}")
	

# Function for frequent use for houlrly data
# the Function is take from this source.
#https://open-meteo.com/en/docs/historical-weather-api#start_date=1950-01-01&end_date=2023-12-31&hourly=&daily=temperature_2m_mean&timezone=Europe%2FBerlin&models=era5_seamless
def Hourly_open_meteo_data(start_date:str, end_date:str, lat:list, long:list, variable):
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
	"latitude": lat,
	"longitude": long,
	"start_date":start_date,
	"end_date": end_date,
	"timezone": "Europe/Berlin",
	"hourly": variable,
	"models": ["era5_seamless"]
	}
	responses = openmeteo.weather_api(url, params=params)
	response = responses[0]
	# Process daily data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_snow_depth = hourly.Variables(0).ValuesAsNumpy()

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["open_meteo"] = hourly_snow_depth

	hourly_dataframe = pd.DataFrame(data = hourly_data)

	if hourly_dataframe.empty:
		return print("The daily limit reached")
	return hourly_dataframe



# plots
def plot_my_bargraph(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue',xtick = False):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    if xtick == True:
        plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.tight_layout()
    plt.show()



def plot_my_bargraph_withnumbers(length:int,breath:int,xaxis,yaxis,title:str,xlabel:str,ylabel:str, widthv=0.5, colorv='blue'):
    plt.figure(figsize=(length, breath))
    plt.bar(xaxis, yaxis, width = widthv , color=colorv)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis='y')
    for i, value in enumerate(yaxis):
        plt.text(xaxis[i], value + 1, str(value), ha='center', fontsize=9)
    plt.tight_layout()
    plt.xticks(xaxis,rotation=45,fontsize=10)
    plt.show()

### Cleaning the data set of DWD,

In [4]:
Dwd_50_year_data = pd.read_csv(r'.\Taglich\50_years_data.csv',sep= ';')

In [5]:
new_column = {'MESS_DATUM':'date','  FX':'Maximum wind peak ','  FM':'Daily average wind speed m/s',' RSK':'daily rainfall','RSKF':'daily form of precipitation',' TMK':'Daily average temperature',}
drop = ['QN_3','QN_4',' SDK','SHK_TAG', '  NM', ' VPM', '  PM', ' UPM', ' TNK', ' TGK', 'eor', ' TXK','STATIONS_ID']
Dwd_50_year_data.drop(columns=drop,inplace=True)

In [6]:
drop = ['QN_3','QN_4',' SDK','SHK_TAG', '  NM', ' VPM', '  PM', ' UPM', ' TNK', ' TGK', 'eor', ' TXK','STATIONS_ID']
Dwd_50_year_data.rename(columns=new_column, inplace=True)
Dwd_50_year_data['date'] = pd.to_datetime(Dwd_50_year_data['date'], format='%Y%m%d')
Dwd_50_year_data.apply(pd.to_numeric)
Dwd_50_year_data.replace(to_replace= -999.0, value=0,inplace=True)
Dwd_50_year_data.replace(to_replace= -999, value=0,inplace=True)
Dwd_50_year_data.set_index('date',inplace=True)
Dwd_50_year_data

,Maximum wind peak,Daily average wind speed m/s,daily rainfall,daily form of precipitation,Daily average temperature
date,,,,,
1896-01-01,0.0,0.0,0.0,0,-6.7
1896-01-02,0.0,0.0,2.1,0,-2.2
1896-01-03,0.0,0.0,1.8,0,0.1
1896-01-04,0.0,0.0,0.9,0,-4.0
1896-01-05,0.0,0.0,0.0,0,-3.2
...,...,...,...,...,...
2023-12-27,32.5,13.7,1.4,8,-1.2
2023-12-28,33.0,22.6,1.3,8,0.8
2023-12-29,36.9,22.9,21.7,4,1.2


### Calclutating the accuracy of model "Era5_seamless" using RMSE and Co_relation

### For tempreature

In [7]:
open_meto_50_years_Temp_dataframe = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.79], [10.62],"temperature_2m_mean",["era5_seamless"])
calculate_accuracy( Dwd_50_year_data["1950-01-01":"2023-12-31"]["Daily average temperature"],open_meto_50_years_Temp_dataframe["open_meteo"])

Mean Bias Error (MBE): nan
Root Mean Square Error (RMSE): 2.395438885439637
Correlation: 0.9515450721927863


### Precipitation

In [10]:
open_meto_50_years_precipitation = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.79], [10.62],"precipitation_sum",["era5_seamless"])
calculate_accuracy( Dwd_50_year_data["1950-01-01":"2023-12-31"]["daily form of precipitation"],open_meto_50_years_precipitation["open_meteo"])

Mean Bias Error (MBE): nan
Root Mean Square Error (RMSE): 4.668683925941836
Correlation: 0.2688971588581231


### RainFall

In [12]:
open_meto_50_years_precipitation = daily_open_meteo_data("1950-01-01", "2023-12-31",[51.79], [10.62],"rain_sum",["era5_seamless"])
calculate_accuracy( Dwd_50_year_data["1950-01-01":"2023-12-31"]["daily rainfall"],open_meto_50_years_precipitation["open_meteo"])

Mean Bias Error (MBE): nan
Root Mean Square Error (RMSE): 6.880847891969073
Correlation: 0.6265610455756389


## Calclutating the accuracy of model "Cerra" using RMSE and Co_relation
### for the cerra model the data is only avaialable between 1985 to June 2021 


### Tempreture

### Handling the error one Nan value and replacing it with 0

In [24]:
C_open_meto_50_years_Temp_dataframe = daily_open_meteo_data("1986-01-01", "2020-12-31",[51.79], [10.62],"temperature_2m_mean",["cerra"])

In [25]:
C_open_meto_50_years_Temp_dataframe.isna().sum()

date          0
open_meteo    1
dtype: int64

In [26]:
C_open_meto_50_years_Temp_dataframe.fillna(0,inplace=True)

In [27]:
calculate_accuracy( Dwd_50_year_data["1986-01-01":"2020-12-31"]["Daily average temperature"],C_open_meto_50_years_Temp_dataframe["open_meteo"])

Mean Bias Error (MBE): nan
Root Mean Square Error (RMSE): 1.8961551016485763
Correlation: 0.9647151339361298


### Precipitation

In [39]:
C_open_meto_50_years_precipitation = daily_open_meteo_data("1986-01-01", "2020-12-31",[51.79], [10.62],"precipitation_sum",["cerra"])
C_open_meto_50_years_precipitation.fillna(0,inplace=True) # to be sure it do not contain NAN values
calculate_accuracy( Dwd_50_year_data["1986-01-01":"2020-12-31"]["daily form of precipitation"],C_open_meto_50_years_precipitation["open_meteo"])

Mean Bias Error (MBE): nan
Root Mean Square Error (RMSE): 6.118380530617456
Correlation: 0.33754814932570426


In [40]:
C_open_meto_50_years_rainfall = daily_open_meteo_data("1986-01-01", "2020-12-31",[51.79], [10.62],"rain_sum",["cerra"])
C_open_meto_50_years_rainfall.fillna(0,inplace=True) # to be sure it do not contain NAN values
calculate_accuracy( Dwd_50_year_data["1986-01-01":"2020-12-31"]["daily rainfall"],C_open_meto_50_years_rainfall["open_meteo"])

RetryError: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Max retries exceeded with url: /v1/archive?latitude=51.79&longitude=10.62&start_date=1986-01-01&end_date=2020-12-31&timezone=Europe%2FBerlin&daily=rain_sum&models=cerra&format=flatbuffers (Caused by ResponseError('too many 500 error responses'))